In [11]:
import py_entitymatching as em

invoices = em.read_csv_metadata('./dataset/invoice_items.csv')
po = em.read_csv_metadata('./dataset/purchase_order_items.csv')


print(invoices)
print(po)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


                           invoice_item_id  \
0     d81458ca-b200-4a9d-b48e-77f56e54b899   
1     689e6556-2c76-4a2f-baa0-e7791e2bdccf   
2     d5c9d034-a1d4-4b6a-91be-5bad8b82a12b   
3     6f07f920-15cf-4fc7-9704-0f549e002fb9   
4     f9fd564b-5cab-4a5b-8d86-cbd09f1c3960   
...                                    ...   
9503  9513c53e-6ac6-4317-a04d-a86a1b809531   
9504  96c10b99-6bc8-452b-a2bd-44f609952a95   
9505  d3d21f8f-fdb4-4d83-b19c-f68ae7cdfb4e   
9506  b4dfe3ba-636a-4c98-824b-53d421c8823d   
9507  5afa63d8-dee4-4544-b81c-d4147f9e906d   

                                invoice_id description  \
0     58006064-8965-4e66-9b91-c527e9a891e3         NaN   
1     2f820fdf-9386-4dc9-96e6-5653913f3dcf         NaN   
2     b49ffe25-2edc-40b7-9dd7-1fd0554d7734         NaN   
3     565aef0f-4d84-42ab-b184-8aad2f18e811         NaN   
4     6a831136-07dd-4c9d-9bd2-c6a17e4ed42f         NaN   
...                                    ...         ...   
9503  1d77cfc7-d358-4c6b-89ab-5591243dd8c

In [12]:
## Drop duplicates
print('Before dropping duplicates:')
print(invoices.duplicated().sum())
print(po.duplicated().sum())

invoices = invoices.drop_duplicates()
po = po.drop_duplicates()
po = po.reset_index()

print('After dropping duplicates:')
print(invoices.duplicated().sum())
print(po.duplicated().sum())

# print(invoices.isnull().sum())
# print(po.isnull().sum())

## replace missing values
invoices['unit'] = invoices['unit'].fillna('')
invoices['quantity'] = invoices['quantity'].fillna(0)
invoices['unit_price'] = invoices['unit_price'].fillna(0)
invoices['total_price_before_vat'] = invoices['total_price_before_vat'].fillna(invoices['unit_price']*invoices['quantity'])

print(invoices.isnull().sum())
print(po.isnull().sum())

Before dropping duplicates:
0
5
After dropping duplicates:
0
0
invoice_item_id              0
invoice_id                   0
description               9507
item_name                    0
unit                         0
quantity                     0
unit_price                   0
vat_percentage              28
vat_amount                   0
promotion                    0
debit_account             9508
credit_account            9508
total_price_before_vat       0
total_price_after_vat        0
line_number                  0
item_code                 6071
tax_account               9508
discount_percentage          0
discount_amount              0
extra                     9508
extra_name                9508
extra_price_after_vat     9508
extra_price_before_vat    9508
extra_vat_amount          9508
buyer_product_code         708
vat_display               9508
created_at                   0
updated_at                   0
batch_number              9504
expire_date               9508
vat_typ

In [13]:
## Stemming for Vietnamese language
import re
from pyvi import ViTokenizer

def convert_lower_case(text):
    return text.lower() if type(text) == str else str(text).lower()

def remove_punctuation(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace('_', ' ')
    return text

s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
	s = ''
	for c in input_str:
		if c in s1:
			s += s0[s1.index(c)]
		else:
			s += c
	return s

def stemming(text):
    text = re.split(r'(\d+)', text)
    text = ' '.join(text)
    return ViTokenizer.tokenize(text)

def preprocess(text):
    text = convert_lower_case(text)
    text = stemming(text)
    text = remove_punctuation(text)
    text = remove_accents(text)
    return text

def preprocess_unit(text):
    text = convert_lower_case(text)
    text = stemming(text)
    text = remove_punctuation(text)
    text = remove_accents(text)
    return text

def dict_conv(text):
    return eval(text) if type(text) == str else text

invoices['item_name'] = invoices['item_name'].apply(preprocess)
po['item_name'] = po['item_name'].apply(preprocess)

invoices['unit'] = invoices['unit'].apply(preprocess_unit)
po['unit'] = po['unit'].apply(preprocess_unit)

po['unit_prices'] = po['unit_prices'].apply(dict_conv)


print(invoices['item_name'])
print(invoices['unit'])
print(po['item_name'])
print(po['unit'])

0       sua uong len men yakult  65 mlx 5 
1       sua uong len men yakult  65 mlx 5 
2       sua uong len men yakult  65 mlx 5 
3       sua uong len men yakult  65 mlx 5 
4       sua uong len men yakult  65 mlx 5 
                       ...                
9503                     banh su kem 66 gr
9504                 banh bun ca phe 80 gr
9505                banh bong lan tam giac
9506             banh sung trau pho mai \n
9507       banh croissant trung muoi 60 gr
Name: item_name, Length: 9508, dtype: object
0       chai
1       chai
2       chai
3       chai
4       chai
        ... 
9503     hop
9504     cai
9505     cai
9506     cai
9507     cai
Name: unit, Length: 9508, dtype: object
0       yakult sua chua uong len men  65 mlx 5 chai   1 loc
1       yakult sua chua uong len men  65 mlx 5 chai   1 loc
2       yakult sua chua uong len men  65 mlx 5 chai   1 loc
3       yakult sua chua uong len men  65 mlx 5 chai   1 loc
4       yakult sua chua uong len men  65 mlx 5 chai   1 loc

In [14]:
# ## Blocking
# ## Overlap blocker
# ob = em.OverlapBlocker()
# em.set_key(invoices, 'invoice_item_id')
# em.set_key(po, 'purchase_order_item_id')

# C = ob.block_tables(invoices, po, 'item_name', 'item_name', word_level=True, overlap_size=1, l_output_attrs=['item_name', 'unit', 'quantity', 'unit_price', 'vat_amount', 'total_price_before_vat'], r_output_attrs=['item_name', 'unit', 'quantity', 'unit_price', 'vat_amount', 'total_price_before_vat', 'unit_prices'], show_progress=True)

# C

## Blocking
## Overlap blocker using q-gram
ob = em.OverlapBlocker()
em.set_key(invoices, 'invoice_item_id')
em.set_key(po, 'purchase_order_item_id')

C = ob.block_tables(invoices, po, 'item_name', 'item_name', word_level=False, q_val=3, overlap_size=7, l_output_attrs=['item_name', 'unit', 'quantity', 'unit_price', 'vat_amount', 'total_price_before_vat'], r_output_attrs=['item_name', 'unit', 'quantity', 'unit_price', 'vat_amount', 'total_price_before_vat', 'unit_prices'], show_progress=True)

C

c:\Users\minx\anaconda3\Lib\site-packages\py_entitymatching\blocker\overlap_blocker.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l_df[l_dummy_overlap_attr] = l_df[l_overlap_attr]
c:\Users\minx\anaconda3\Lib\site-packages\py_entitymatching\blocker\overlap_blocker.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_df[r_dummy_overlap_attr] = r_df[r_overlap_attr]
c:\Users\minx\anaconda3\Lib\site-packages\py_entitymatching\blocker\overlap_blocker.py:615: SettingWithCopyWarning: 
A value is trying t

,_id,ltable_invoice_item_id,rtable_purchase_order_item_id,ltable_item_name,ltable_unit,ltable_quantity,ltable_unit_price,ltable_vat_amount,ltable_total_price_before_vat,rtable_item_name,rtable_unit,rtable_quantity,rtable_unit_price,rtable_vat_amount,rtable_total_price_before_vat,rtable_unit_prices
0,0,d81458ca-b200-4a9d-b48e-77f56e54b899,b36023e3-810d-4cec-b8ba-c42a7abb7593,sua uong len men yakult 65 mlx 5,chai,50.0,4200.0,16800.0,210000.0,yakult sua chua uong len men 65 mlx 5 chai 1 loc,loc,10,21000.0,16800.0,210000.0,"[{'unit': 'CHAI', 'unit_price': 4200, 'conversion_rate': 0.2}, {'unit': 'CHAI', 'unit_price': 42..."
1,1,689e6556-2c76-4a2f-baa0-e7791e2bdccf,b36023e3-810d-4cec-b8ba-c42a7abb7593,sua uong len men yakult 65 mlx 5,chai,100.0,4200.0,33600.0,420000.0,yakult sua chua uong len men 65 mlx 5 chai 1 loc,loc,10,21000.0,16800.0,210000.0,"[{'unit': 'CHAI', 'unit_price': 4200, 'conversion_rate': 0.2}, {'unit': 'CHAI', 'unit_price': 42..."
2,2,d5c9d034-a1d4-4b6a-91be-5bad8b82a12b,b36023e3-810d-4cec-b8ba-c42a7abb7593,sua uong len men yakult 65 mlx 5,chai,100.0,4200.0,33600.0,420000.0,yakult sua chua uong len men 65 mlx 5 chai 1 loc,loc,10,21000.0,16800.0,210000.0,"[{'unit': 'CHAI', 'unit_price': 4200, 'conversion_rate': 0.2}, {'unit': 'CHAI', 'unit_price': 42..."
3,3,6f07f920-15cf-4fc7-9704-0f549e002fb9,b36023e3-810d-4cec-b8ba-c42a7abb7593,sua uong len men yakult 65 mlx 5,chai,100.0,4200.0,33600.0,420000.0,yakult sua chua uong len men 65 mlx 5 chai 1 loc,loc,10,21000.0,16800.0,210000.0,"[{'unit': 'CHAI', 'unit_price': 4200, 'conversion_rate': 0.2}, {'unit': 'CHAI', 'unit_price': 42..."
4,4,f9fd564b-5cab-4a5b-8d86-cbd09f1c3960,b36023e3-810d-4cec-b8ba-c42a7abb7593,sua uong len men yakult 65 mlx 5,chai,250.0,4200.0,84000.0,1050000.0,yakult sua chua uong len men 65 mlx 5 chai 1 loc,loc,10,21000.0,16800.0,210000.0,"[{'unit': 'CHAI', 'unit_price': 4200, 'conversion_rate': 0.2}, {'unit': 'CHAI', 'unit_price': 42..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982382,7982382,debbde39-bf4a-4fa8-a2a8-bb06d5541332,972948dc-b9b1-4059-b0f5-f89d85398a0c,cha ca que kieu nhat vi truyen thong 40 g goi 400 g hop mw 629,goi,10.0,8000.0,6400.0,80000.0,ck nguyen lieu bmi xuc xich 70 g 1 cai,cai,5,2900.0,1160.0,14500.0,"[{'unit': 'CAI', 'unit_price': 2900, 'conversion_rate': 1}]"
7982383,7982383,029e5e9e-de66-4a7b-9205-0f7eb11b7e1d,972948dc-b9b1-4059-b0f5-f89d85398a0c,xuc xich pho mai 900 g hs 673,bich,6.0,102312.0,49110.0,613872.0,ck nguyen lieu bmi xuc xich 70 g 1 cai,cai,5,2900.0,1160.0,14500.0,"[{'unit': 'CAI', 'unit_price': 2900, 'conversion_rate': 1}]"
7982384,7982384,85eb699f-be0c-44d1-a81b-2fae7f2098bb,972948dc-b9b1-4059-b0f5-f89d85398a0c,cha ca que kieu nhat vi truyen thong 40 g goi 400 g hop mw 629,goi,10.0,8000.0,6400.0,80000.0,ck nguyen lieu bmi xuc xich 70 g 1 cai,cai,5,2900.0,1160.0,14500.0,"[{'unit': 'CAI', 'unit_price': 2900, 'conversion_rate': 1}]"
7982385,7982385,3c5e3e22-1f1d-4b8d-aa1e-ff5fa5b37beb,972948dc-b9b1-4059-b0f5-f89d85398a0c,xuc xich deli vi ga sa ot 700 g 26 mm 70 g hs 841,bich,10.0,82000.0,65600.0,820000.0,ck nguyen lieu bmi xuc xich 70 g 1 cai,cai,5,2900.0,1160.0,14500.0,"[{'unit': 'CAI', 'unit_price': 2900, 'conversion_rate': 1}]"


In [15]:
# ## Similarity calculation to calculate similarity between two names
# def similarity_calculation(x, y):
#     ## Cosine similarity
#     x, y = x.split(), y.split()
#     x = set(x)
#     y = set(y)
#     inter = x.intersection(y)
#     return len(inter) / (len(x) * len(y)) ** 0.5

## Using q-gram to calculate similarity between two names

# ## Using overlap similarity to evaluate the similarity of unit
# def overlap_similarity(x, y):
#     x, y = x.split(), y.split()
#     x = set(x)
#     y = set(y)
#     inter = x.intersection(y)
#     return len(inter) / max(1, min(len(x), len(y)))

# ## Check unit, if not match, convert to the same unit using unit_prices
# def check_unit(x, y, unit_prices):
#     if x == y: return 1
#     else:
#         for conv in unit_prices:
#             if preprocess(conv['unit']) == x:

#                 return 1
            
def similarity_calculation(x, y):
    x, y = x.split(), y.split()
    x = set(x)
    y = set(y)
    inter = x.intersection(y)
    return len(inter) / max(1, min(len(x), len(y)))


## Define a function to calculate similarity of unit price, total price before vat, vat amount
def price_similarity(x, y):
    x, y = float(x), float(y)
    if x == 0 and y == 0: return 0
    return 1 if abs(x-y)/max(abs(x), abs(y)) < 0.05 else 0

## Define a function to calculate similarity of quantity
def quantity_similarity(x, y):
    x, y = float(x), float(y)
    if max(x, y) == 0: return 0
    return abs(min(x, y) / max(x, y))

def calculate_unit_price(x, y, unit_prices, quantity, unit, unit_price):
    if x == y: return 1
    elif type(unit_prices) == list:
        for conv in unit_prices:
            if 'unit' not in conv.keys():
                continue
            elif preprocess(conv['unit']) == x:
                quantity = quantity / conv['conversion_rate']
                unit = conv['unit']
                unit_price = conv['unit_price']
                return 1
    return 0

## Add new column to store item name similarity
C['name_similarity'] = C.apply(lambda x: similarity_calculation(x['ltable_item_name'], x['rtable_item_name']), axis=1)
## Add new column to store unit similarity
C['unit_similarity'] = C.apply(lambda x: calculate_unit_price(x['ltable_unit'], x['rtable_unit'], x['rtable_unit_prices'], x['rtable_quantity'], x['rtable_unit'], x['rtable_unit_price']), axis=1)
## Add new column to store unit price similarity
C['unit_price_similarity'] = C.apply(lambda x: price_similarity(x['ltable_unit_price'], x['rtable_unit_price']), axis=1)
## Add new column to store quantity similarity
C['quantity_similarity'] = C.apply(lambda x: quantity_similarity(x['ltable_quantity'], x['rtable_quantity']), axis=1)
## Add new column to store total price before vat similarity
C['total_price_before_vat_similarity'] = C.apply(lambda x: price_similarity(x['ltable_total_price_before_vat'], x['rtable_total_price_before_vat']), axis=1)
## Add new column to store vat amount similarity
C['vat_amount_similarity'] = C.apply(lambda x: price_similarity(x['ltable_vat_amount'], x['rtable_vat_amount']), axis=1)


## Calculate similarity
# for i in range(len(C)):
#     C.loc[i, 'name_similarity'] = similarity_calculation(C['ltable_item_name'].iloc[i], C['rtable_item_name'].iloc[i])
#     C.loc[i, 'unit_similarity'] = overlap_similarity(C['ltable_unit'].iloc[i], C['rtable_unit'].iloc[i])
#     C.loc[i, 'unit_price_similarity'] = price_similarity(C['ltable_unit_price'].iloc[i], C['rtable_unit_price'].iloc[i])
#     C.loc[i, 'quantity_similarity'] = quantity_similarity(C['ltable_quantity'].iloc[i], C['rtable_quantity'].iloc[i])
#     C.loc[i, 'total_price_before_vat_similarity'] = price_similarity(C['ltable_total_price_before_vat'].iloc[i], C['rtable_total_price_before_vat'].iloc[i])
#     C.loc[i, 'vat_amount_similarity'] = price_similarity(C['ltable_vat_amount'].iloc[i], C['rtable_vat_amount'].iloc[i])

    # C['name_similarity'][i] = similarity_calculation(C['ltable_item_name'][i], C['rtable_item_name'][i])
    # C['unit_similarity'][i] = overlap_similarity(C['ltable_unit'][i], C['rtable_unit'][i])
    # C['unit_price_similarity'][i] = price_similarity(C['ltable_unit_price'][i], C['rtable_unit_price'][i])
    # C['quantity_similarity'][i] = quantity_similarity(C['ltable_quantity'][i], C['rtable_quantity'][i])
    # C['total_price_before_vat_similarity'][i] = price_similarity(C['ltable_total_price_before_vat'][i], C['rtable_total_price_before_vat'][i])
    # C['vat_amount_similarity'][i] = price_similarity(C['ltable_vat_amount'][i], C['rtable_vat_amount'][i])
print(C.isnull().sum())
C

_id                                  0
ltable_invoice_item_id               0
rtable_purchase_order_item_id        0
ltable_item_name                     0
ltable_unit                          0
ltable_quantity                      0
ltable_unit_price                    0
ltable_vat_amount                    0
ltable_total_price_before_vat        0
rtable_item_name                     0
rtable_unit                          0
rtable_quantity                      0
rtable_unit_price                    0
rtable_vat_amount                    0
rtable_total_price_before_vat        0
rtable_unit_prices                   0
name_similarity                      0
unit_similarity                      0
unit_price_similarity                0
quantity_similarity                  0
total_price_before_vat_similarity    0
vat_amount_similarity                0
dtype: int64


,_id,ltable_invoice_item_id,rtable_purchase_order_item_id,ltable_item_name,ltable_unit,ltable_quantity,ltable_unit_price,ltable_vat_amount,ltable_total_price_before_vat,rtable_item_name,...,rtable_unit_price,rtable_vat_amount,rtable_total_price_before_vat,rtable_unit_prices,name_similarity,unit_similarity,unit_price_similarity,quantity_similarity,total_price_before_vat_similarity,vat_amount_similarity
0,0,d81458ca-b200-4a9d-b48e-77f56e54b899,b36023e3-810d-4cec-b8ba-c42a7abb7593,sua uong len men yakult 65 mlx 5,chai,50.0,4200.0,16800.0,210000.0,yakult sua chua uong len men 65 mlx 5 chai 1 loc,...,21000.0,16800.0,210000.0,"[{'unit': 'CHAI', 'unit_price': 4200, 'conversion_rate': 0.2}, {'unit': 'CHAI', 'unit_price': 42...",1.000,1,0,0.200000,1,1
1,1,689e6556-2c76-4a2f-baa0-e7791e2bdccf,b36023e3-810d-4cec-b8ba-c42a7abb7593,sua uong len men yakult 65 mlx 5,chai,100.0,4200.0,33600.0,420000.0,yakult sua chua uong len men 65 mlx 5 chai 1 loc,...,21000.0,16800.0,210000.0,"[{'unit': 'CHAI', 'unit_price': 4200, 'conversion_rate': 0.2}, {'unit': 'CHAI', 'unit_price': 42...",1.000,1,0,0.100000,0,0
2,2,d5c9d034-a1d4-4b6a-91be-5bad8b82a12b,b36023e3-810d-4cec-b8ba-c42a7abb7593,sua uong len men yakult 65 mlx 5,chai,100.0,4200.0,33600.0,420000.0,yakult sua chua uong len men 65 mlx 5 chai 1 loc,...,21000.0,16800.0,210000.0,"[{'unit': 'CHAI', 'unit_price': 4200, 'conversion_rate': 0.2}, {'unit': 'CHAI', 'unit_price': 42...",1.000,1,0,0.100000,0,0
3,3,6f07f920-15cf-4fc7-9704-0f549e002fb9,b36023e3-810d-4cec-b8ba-c42a7abb7593,sua uong len men yakult 65 mlx 5,chai,100.0,4200.0,33600.0,420000.0,yakult sua chua uong len men 65 mlx 5 chai 1 loc,...,21000.0,16800.0,210000.0,"[{'unit': 'CHAI', 'unit_price': 4200, 'conversion_rate': 0.2}, {'unit': 'CHAI', 'unit_price': 42...",1.000,1,0,0.100000,0,0
4,4,f9fd564b-5cab-4a5b-8d86-cbd09f1c3960,b36023e3-810d-4cec-b8ba-c42a7abb7593,sua uong len men yakult 65 mlx 5,chai,250.0,4200.0,84000.0,1050000.0,yakult sua chua uong len men 65 mlx 5 chai 1 loc,...,21000.0,16800.0,210000.0,"[{'unit': 'CHAI', 'unit_price': 4200, 'conversion_rate': 0.2}, {'unit': 'CHAI', 'unit_price': 42...",1.000,1,0,0.040000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982382,7982382,debbde39-bf4a-4fa8-a2a8-bb06d5541332,972948dc-b9b1-4059-b0f5-f89d85398a0c,cha ca que kieu nhat vi truyen thong 40 g goi 400 g hop mw 629,goi,10.0,8000.0,6400.0,80000.0,ck nguyen lieu bmi xuc xich 70 g 1 cai,...,2900.0,1160.0,14500.0,"[{'unit': 'CAI', 'unit_price': 2900, 'conversion_rate': 1}]",0.100,0,0,0.500000,0,0
7982383,7982383,029e5e9e-de66-4a7b-9205-0f7eb11b7e1d,972948dc-b9b1-4059-b0f5-f89d85398a0c,xuc xich pho mai 900 g hs 673,bich,6.0,102312.0,49110.0,613872.0,ck nguyen lieu bmi xuc xich 70 g 1 cai,...,2900.0,1160.0,14500.0,"[{'unit': 'CAI', 'unit_price': 2900, 'conversion_rate': 1}]",0.375,0,0,0.833333,0,0
7982384,7982384,85eb699f-be0c-44d1-a81b-2fae7f2098bb,972948dc-b9b1-4059-b0f5-f89d85398a0c,cha ca que kieu nhat vi truyen thong 40 g goi 400 g hop mw 629,goi,10.0,8000.0,6400.0,80000.0,ck nguyen lieu bmi xuc xich 70 g 1 cai,...,2900.0,1160.0,14500.0,"[{'unit': 'CAI', 'unit_price': 2900, 'conversion_rate': 1}]",0.100,0,0,0.500000,0,0
7982385,7982385,3c5e3e22-1f1d-4b8d-aa1e-ff5fa5b37beb,972948dc-b9b1-4059-b0f5-f89d85398a0c,xuc xich deli vi ga sa ot 700 g 26 mm 70 g hs 841,bich,10.0,82000.0,65600.0,820000.0,ck nguyen lieu bmi xuc xich 70 g 1 cai,...,2900.0,1160.0,14500.0,"[{'unit': 'CAI', 'unit_price': 2900, 'conversion_rate': 1}]",0.400,0,0,0.500000,0,0


In [16]:
## Store the result
C.to_csv('candidate_set.csv', index=False)